<a href="https://colab.research.google.com/github/Juliana-de-Andrade/CP02Part1/blob/main/CP02Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports
import numpy as np
import pandas as pd
import zipfile
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import os

ExERCÍCIO 1 – CLASSIFICAÇÃO MULTICLASSE
Dataset: Wine Dataset (UCI)
1. Treinar uma rede neural em Keras para classificar vinhos em 3 classes.
- Configuração mínima: 2 camadas ocultas com 32 neurônios cada, função de ativação ReLU.
- Camada de saída com 3 neurônios, função de ativação Softmax.
- Função de perda: categorical_crossentropy.
- Otimizador: Adam.
2. Comparar os resultados com um modelo do scikit-learn (RandomForestClassifier ou
LogisticRegression).
3. Registrar métricas de acurácia e discutir qual modelo teve melhor desempenho

In [ ]:
# Extrair o ZIP
zip_path = '/content/wine.zip'
extract_path = '/content/wine_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verificar arquivos extraídos
files = os.listdir(extract_path)
print("Arquivos extraídos:", files)

# Procurar arquivo .data
data_file = None
for f in files:
    if f.endswith('.data'):
        data_file = f
        break

if data_file is None:
    raise FileNotFoundError("Arquivo .data não encontrado dentro do ZIP")

dataset_path = os.path.join(extract_path, data_file)

In [7]:
colunas = [
    'Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium',
    'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
    'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline'
]

data = pd.read_csv(dataset_path, header=None, names=colunas)
print("\nVisualização inicial do dataset:")
print(data.head())

# Separar features (X) e rótulos (y)
X = data.iloc[:, 1:].values   # Todas as colunas exceto 'Class'
y = data.iloc[:, 0].values    # Coluna 'Class'

# Converter rótulos para categorias (0,1,2)
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)  # para Keras



Visualização inicial do dataset:
   Class  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium  \
0      1    14.23        1.71  2.43               15.6        127   
1      1    13.20        1.78  2.14               11.2        100   
2      1    13.16        2.36  2.67               18.6        101   
3      1    14.37        1.95  2.50               16.8        113   
4      1    13.24        2.59  2.87               21.0        118   

   Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   
3           3.85        3.49                  0.24             2.18   
4           2.80        2.69                  0.39             1.82   

   Color intensity   Hue  OD280/OD315 of diluted wines  Proline  
0             5.64  1.04                          3.92    

In [8]:
#Divisão Treino/Teste

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y_categorical
)

# Escalonar os dados (importante para NN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
#Rede Neural (Keras)

model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=8,
    verbose=0,  # mude para 1 se quiser ver o progresso
    validation_split=0.1
)

# Avaliar no conjunto de teste
loss, acc_nn = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Acurácia da Rede Neural (Keras): {acc_nn:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



✅ Acurácia da Rede Neural (Keras): 1.0000


In [10]:
#Random Forest (Scikit-learn)

# Converter rótulos de volta para inteiros para RF
y_train_rf = np.argmax(y_train, axis=1)
y_test_rf = np.argmax(y_test, axis=1)

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train_rf)
y_pred_rf = rf.predict(X_test)

acc_rf = accuracy_score(y_test_rf, y_pred_rf)
print(f"✅ Acurácia do RandomForestClassifier: {acc_rf:.4f}")

# Relatório de classificação
print("\n=== Classification Report (Random Forest) ===")
print(classification_report(y_test_rf, y_pred_rf, target_names=encoder.classes_.astype(str)))


✅ Acurácia do RandomForestClassifier: 1.0000

=== Classification Report (Random Forest) ===
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00        14
           3       1.00      1.00      1.00        10

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



In [11]:
#Comparação Final
# ===============================
if acc_nn > acc_rf:
    melhor = "Rede Neural (Keras)"
elif acc_nn < acc_rf:
    melhor = "Random Forest"
else:
    melhor = "Empate"

print("\n==============================")
print(f"Melhor desempenho: {melhor}")
print(f"Acurácia Rede Neural: {acc_nn:.4f}")
print(f"Acurácia Random Forest: {acc_rf:.4f}")
print("==============================")


Melhor desempenho: Empate
Acurácia Rede Neural: 1.0000
Acurácia Random Forest: 1.0000


EXERCÍCIO 2 – REGRESSÃO
Dataset: California Housing Dataset (Scikit-learn)
1. Treinar uma rede neural em Keras para prever o valor médio das casas.
- Configuração mínima: 3 camadas ocultas com 64, 32 e 16 neurônios, função de ativação ReLU.
- Camada de saída com 1 neurônio, função de ativação Linear.
- Função de perda: mse.
- Otimizador: Adam.
2. Comparar os resultados com um modelo do scikit-learn (LinearRegression ou
RandomForestRegressor).
3. Registrar métricas de erro (RMSE ou MAE) e discutir qual modelo teve melhor desempenho

In [ ]:
# Carregar e preparar os dados
print("Carregando o dataset California Housing...")
housing = fetch_california_housing()
X, y = housing.data, housing.target

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
# Escalar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Shape dos dados de treino: {X_train.shape}")
print(f"Shape dos dados de teste: {X_test.shape}")
print(f"Nomes das features: {housing.feature_names}")

In [ ]:
# 2. Treinar modelo Keras
print("\n" + "="*50)
print("Treinando Rede Neural Keras...")
print("="*50)

def create_keras_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='linear')
    ])

    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    return model

# Criar e treinar o modelo
keras_model = create_keras_model()

print("Arquitetura da rede neural:")
keras_model.summary()

# Callback para early stopping
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True)

In [ ]:
# Treinar o modelo
history = keras_model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1)

# Fazer previsões
y_pred_keras = keras_model.predict(X_test_scaled).flatten()

# 3. Treinar modelos scikit-learn
print("\n" + "="*50)
print("Treinando modelos scikit-learn...")
print("="*50)

# Linear Regression
print("Treinando Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)

# Random Forest Regressor
print("Treinando Random Forest Regressor...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)  # Random Forest não precisa de scaling
y_pred_rf = rf_model.predict(X_test)

In [ ]:
# 4. Avaliar e comparar modelos
print("\n" + "="*50)
print("COMPARAÇÃO DOS MODELOS")
print("="*50)

def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)

    print(f"\n{model_name}:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  MSE:  {mse:.4f}")

    return rmse, mae

# Avaliar todos os modelos
results = {}
results['Keras NN'] = evaluate_model(y_test, y_pred_keras, "Keras Neural Network")
results['Linear Regression'] = evaluate_model(y_test, y_pred_lr, "Linear Regression")
results['Random Forest'] = evaluate_model(y_test, y_pred_rf, "Random Forest")

# 5. Visualizar resultados
plt.figure(figsize=(15, 5))

# Gráfico 1: Comparação de previsões vs valores reais
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred_keras, alpha=0.5, label='Keras NN')
plt.scatter(y_test, y_pred_rf, alpha=0.5, label='Random Forest')
plt.scatter(y_test, y_pred_lr, alpha=0.5, label='Linear Regression')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valor Real')
plt.ylabel('Valor Previsto')
plt.title('Previsões vs Valores Reais')
plt.legend()

# Gráfico 2: Comparação de RMSE
plt.subplot(1, 3, 2)
models = list(results.keys())
rmse_values = [results[model][0] for model in models]
plt.bar(models, rmse_values, color=['blue', 'orange', 'green'])
plt.title('Comparação do RMSE')
plt.xticks(rotation=45)
plt.ylabel('RMSE')

# Gráfico 3: Comparação de MAE
plt.subplot(1, 3, 3)
mae_values = [results[model][1] for model in models]
plt.bar(models, mae_values, color=['blue', 'orange', 'green'])
plt.title('Comparação do MAE')
plt.xticks(rotation=45)
plt.ylabel('MAE')

plt.tight_layout()
plt.show()

# 6. Análise detalhada
print("\n" + "="*50)
print("ANÁLISE DOS RESULTADOS")
print("="*50)

# Encontrar o melhor modelo baseado no RMSE
best_model_rmse = min(results, key=lambda x: results[x][0])
best_model_mae = min(results, key=lambda x: results[x][1])

print(f"\nMelhor modelo por RMSE: {best_model_rmse} (RMSE = {results[best_model_rmse][0]:.4f})")
print(f"Melhor modelo por MAE: {best_model_mae} (MAE = {results[best_model_mae][1]:.4f})")

# Calcular porcentagem de melhoria
rmse_baseline = results['Linear Regression'][0]
for model_name, (rmse, mae) in results.items():
    if model_name != 'Linear Regression':
        improvement = ((rmse_baseline - rmse) / rmse_baseline) * 100
        print(f"\n{model_name} vs Linear Regression:")
        print(f"  Melhoria no RMSE: {improvement:+.2f}%")

# 7. Visualizar histórico de treinamento da rede neural
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Evolução da Loss durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('Loss (MSE)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Evolução do MAE durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

# 8. Importância das features (Random Forest)
print("\n" + "="*50)
print("IMPORTÂNCIA DAS FEATURES (Random Forest)")
print("="*50)

feature_importance = pd.DataFrame({
    'feature': housing.feature_names,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance)

Comparação de desempenho entre os modelos

In [ ]:
#Conclusão:
#O modelo Random Forest costuma ter uma performance melhor quando se trata de problemas de regressão com dados tabulares, mas a rede neural pode alcançar resultados semelhantes com a arquitetura e tuning adequados. A Regressão Linear serve como uma baseline útil para acompanhamento de possíveis melhorias.